In [ ]:
import tensorflow as tf
import json 
import numpy as np 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

with open('intents.json') as file:
    data = json.load(file)
    
# empty lists to hold training data    
training_sentences = []
training_labels = []
labels = []
responses = []

# iterate over intents and patterns to create training data and labels
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
   # add new tag to labels list if not already present 
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
# calculate number of classes        
num_classes = len(labels)


#--------------Start here preprocessing------------

#lable encoder use converting lables in numric form
# Initialize label encoder and fit it to the training labels
lbl_encoder = LabelEncoder()
#'fit' method trains the algorithm on the training data, after the model is initialized
lbl_encoder.fit(training_labels)

training_labels = lbl_encoder.transform(training_labels)

# Set the maximum vocabulary size, embedding dimension, maximum sequence length, and OOV token
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

# tokenization basically refers to splitting up a larger body of text into smaller words or lines
# Initialize tokenizer object and fit it to the training sentences
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)

# create word index from tokenizer
word_index = tokenizer.word_index

# convert training sentences to sequences and pad them
# Convert the training sentences to sequences and pad them to a maximum length of max_len
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# define and compile model
model = Sequential()
# Add an Embedding layer with the specified vocabulary size, embedding dimension, and sequence length
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
# Add a GlobalAveragePooling1D layer
model.add(GlobalAveragePooling1D())
# Add two Dense layers with ReLU activation
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model with sparse categorical crossentropy loss and Adam optimizer
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

# print model summary
model.summary()

# # Set the number of epochs for training
epochs = 500
# Fit the model to the padded sequences and training labels
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)


# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

import json 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import colorama 
from colorama import Fore, Style, Back
import random
import pickle

colorama.init()
# load intents.json file
with open("intents.json") as file:
    data = json.load(file)

# define function to start chat
def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    # loop to take user input and get bot response
    while True:
        # get user input
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        # Check if user wants to quit
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 19)                323       
                                                                 
Total params: 16,867
Trainable params: 16,867
Non-trainable params: 0
____________________________________________________

3/3 [==============================] - 0s 0s/step - loss: 2.7810 - accuracy: 0.1594
Epoch 74/500
3/3 [==============================] - 0s 4ms/step - loss: 2.7754 - accuracy: 0.1594
Epoch 75/500
3/3 [==============================] - 0s 4ms/step - loss: 2.7690 - accuracy: 0.1884
Epoch 76/500
3/3 [==============================] - 0s 0s/step - loss: 2.7633 - accuracy: 0.1739
Epoch 77/500
3/3 [==============================] - 0s 4ms/step - loss: 2.7563 - accuracy: 0.1739
Epoch 78/500
3/3 [==============================] - 0s 5ms/step - loss: 2.7492 - accuracy: 0.1884
Epoch 79/500
3/3 [==============================] - 0s 9ms/step - loss: 2.7414 - accuracy: 0.1884
Epoch 80/500
3/3 [==============================] - 0s 0s/step - loss: 2.7340 - accuracy: 0.2319
Epoch 81/500
3/3 [==============================] - 0s 0s/step - loss: 2.7268 - accuracy: 0.2464
Epoch 82/500
3/3 [==============================] - 0s 3ms/step - loss: 2.7185 - accuracy: 0.2174
Epoch 83/500
3/3 [===================

3/3 [==============================] - 0s 6ms/step - loss: 1.9760 - accuracy: 0.3768
Epoch 157/500
3/3 [==============================] - 0s 6ms/step - loss: 1.9635 - accuracy: 0.3913
Epoch 158/500
3/3 [==============================] - 0s 5ms/step - loss: 1.9506 - accuracy: 0.3913
Epoch 159/500
3/3 [==============================] - 0s 6ms/step - loss: 1.9398 - accuracy: 0.3768
Epoch 160/500
3/3 [==============================] - 0s 0s/step - loss: 1.9295 - accuracy: 0.3768
Epoch 161/500
3/3 [==============================] - 0s 0s/step - loss: 1.9207 - accuracy: 0.3768
Epoch 162/500
3/3 [==============================] - 0s 3ms/step - loss: 1.9074 - accuracy: 0.3913
Epoch 163/500
3/3 [==============================] - 0s 11ms/step - loss: 1.8961 - accuracy: 0.3913
Epoch 164/500
3/3 [==============================] - 0s 8ms/step - loss: 1.8845 - accuracy: 0.4058
Epoch 165/500
3/3 [==============================] - 0s 0s/step - loss: 1.8727 - accuracy: 0.4203
Epoch 166/500
3/3 [=======

3/3 [==============================] - 0s 5ms/step - loss: 0.9974 - accuracy: 0.7971
Epoch 240/500
3/3 [==============================] - 0s 6ms/step - loss: 0.9858 - accuracy: 0.8261
Epoch 241/500
3/3 [==============================] - 0s 5ms/step - loss: 0.9733 - accuracy: 0.8406
Epoch 242/500
3/3 [==============================] - 0s 0s/step - loss: 0.9639 - accuracy: 0.8406
Epoch 243/500
3/3 [==============================] - 0s 0s/step - loss: 0.9535 - accuracy: 0.8551
Epoch 244/500
3/3 [==============================] - 0s 0s/step - loss: 0.9424 - accuracy: 0.8551
Epoch 245/500
3/3 [==============================] - 0s 0s/step - loss: 0.9327 - accuracy: 0.8406
Epoch 246/500
3/3 [==============================] - 0s 10ms/step - loss: 0.9221 - accuracy: 0.8406
Epoch 247/500
3/3 [==============================] - 0s 10ms/step - loss: 0.9115 - accuracy: 0.8551
Epoch 248/500
3/3 [==============================] - 0s 8ms/step - loss: 0.9011 - accuracy: 0.8696
Epoch 249/500
3/3 [=======

Epoch 322/500
3/3 [==============================] - 0s 5ms/step - loss: 0.3884 - accuracy: 0.9565
Epoch 323/500
3/3 [==============================] - 0s 7ms/step - loss: 0.3842 - accuracy: 0.9565
Epoch 324/500
3/3 [==============================] - 0s 10ms/step - loss: 0.3804 - accuracy: 0.9565
Epoch 325/500
3/3 [==============================] - 0s 8ms/step - loss: 0.3752 - accuracy: 0.9565
Epoch 326/500
3/3 [==============================] - 0s 0s/step - loss: 0.3703 - accuracy: 0.9565
Epoch 327/500
3/3 [==============================] - 0s 0s/step - loss: 0.3647 - accuracy: 0.9710
Epoch 328/500
3/3 [==============================] - 0s 2ms/step - loss: 0.3614 - accuracy: 0.9710
Epoch 329/500
3/3 [==============================] - 0s 11ms/step - loss: 0.3593 - accuracy: 0.9710
Epoch 330/500
3/3 [==============================] - 0s 8ms/step - loss: 0.3549 - accuracy: 0.9710
Epoch 331/500
3/3 [==============================] - 0s 0s/step - loss: 0.3500 - accuracy: 0.9855
Epoch 332/5

3/3 [==============================] - 0s 0s/step - loss: 0.1645 - accuracy: 1.0000
Epoch 405/500
3/3 [==============================] - 0s 4ms/step - loss: 0.1628 - accuracy: 1.0000
Epoch 406/500
3/3 [==============================] - 0s 2ms/step - loss: 0.1610 - accuracy: 1.0000
Epoch 407/500
3/3 [==============================] - 0s 9ms/step - loss: 0.1594 - accuracy: 1.0000
Epoch 408/500
3/3 [==============================] - 0s 0s/step - loss: 0.1580 - accuracy: 1.0000
Epoch 409/500
3/3 [==============================] - 0s 3ms/step - loss: 0.1567 - accuracy: 1.0000
Epoch 410/500
3/3 [==============================] - 0s 10ms/step - loss: 0.1554 - accuracy: 1.0000
Epoch 411/500
3/3 [==============================] - 0s 8ms/step - loss: 0.1541 - accuracy: 1.0000
Epoch 412/500
3/3 [==============================] - 0s 0s/step - loss: 0.1530 - accuracy: 1.0000
Epoch 413/500
3/3 [==============================] - 0s 0s/step - loss: 0.1513 - accuracy: 1.0000
Epoch 414/500
3/3 [========

3/3 [==============================] - 0s 4ms/step - loss: 0.0758 - accuracy: 1.0000
Epoch 487/500
3/3 [==============================] - 0s 6ms/step - loss: 0.0746 - accuracy: 1.0000
Epoch 488/500
3/3 [==============================] - 0s 5ms/step - loss: 0.0737 - accuracy: 1.0000
Epoch 489/500
3/3 [==============================] - 0s 0s/step - loss: 0.0739 - accuracy: 1.0000
Epoch 490/500
3/3 [==============================] - 0s 10ms/step - loss: 0.0736 - accuracy: 1.0000
Epoch 491/500
3/3 [==============================] - 0s 8ms/step - loss: 0.0730 - accuracy: 1.0000
Epoch 492/500
3/3 [==============================] - 0s 0s/step - loss: 0.0719 - accuracy: 1.0000
Epoch 493/500
3/3 [==============================] - 0s 0s/step - loss: 0.0709 - accuracy: 1.0000
Epoch 494/500
3/3 [==============================] - 0s 2ms/step - loss: 0.0699 - accuracy: 1.0000
Epoch 495/500
3/3 [==============================] - 0s 10ms/step - loss: 0.0706 - accuracy: 1.0000
Epoch 496/500
3/3 [======